In [1]:
import sys, os
nb_dir = os.path.dirname(os.getcwd()) if os.path.basename(os.getcwd()) == 'notebooks' else os.getcwd()
proj_root = nb_dir if os.path.isfile(os.path.join(nb_dir, 'config.py')) else os.path.dirname(nb_dir)
if proj_root not in sys.path:
    sys.path.insert(0, proj_root)
print('Using project root:', proj_root)

Using project root: c:\Users\arnav\Desktop\Arnav\ROG\Arnav\Arnav\python\ExoPatternNet


In [ ]:

# Fetch & Preprocess Light Curve
import config
import os, numpy as np, pandas as pd
from config import DEFAULTS

DATA_PRO = "data/processed"
DATA_WIN = "data/windows"
os.makedirs(DATA_WIN, exist_ok=True)

csv_path = [p for p in os.listdir(DATA_PRO) if p.endswith("_flattened.csv")]
assert len(csv_path) >= 1, "Run the preprocessing notebook first."
csv_path = os.path.join(DATA_PRO, csv_path[0])
df = pd.read_csv(csv_path)
flux = df["flux"].to_numpy().astype(np.float32)


In [6]:
def make_windows(series: np.ndarray, win: int = 2048, stride: int = 256):
    T = series.shape[0]
    if T < win:
        return np.zeros((0,1,win), dtype=np.float32)
    out = []
    for s in range(0, T - win + 1, stride):
        seg = series[s:s+win]
        out.append(seg[None, :])  # (1,L)
    X = np.stack(out, axis=0).astype(np.float32)  # (N,1,L)
    return X

WIN = DEFAULTS["window_len"]
STR = DEFAULTS["window_stride"]
X = make_windows(flux, WIN, STR)
X.shape


(162, 1, 2048)

In [7]:
if X.shape[0] < 10:
    print("Warning: very few windows; pick a longer target or reduce win/stride.")

# naive split (later you’ll aggregate multiple stars and split by star)
n = X.shape[0]
n_val = max(1, int(0.2 * n))
X_tr, X_va = X[:-n_val], X[-n_val:]

np.savez_compressed(os.path.join(DATA_WIN, "windows_train.npz"), x=X_tr)
np.savez_compressed(os.path.join(DATA_WIN, "windows_val.npz"),   x=X_va)

with open(os.path.join(DATA_WIN, "config.json"), "w") as f:
    import json
    json.dump({
        "window_len": int(WIN),
        "window_stride": int(STR),
        "norm": "median=1.0 after flatten",
        "mission": DEFAULTS["mission"]
    }, f, indent=2)

print("Wrote:", os.listdir(DATA_WIN))


Wrote: ['config.json', 'windows_train.npz', 'windows_val.npz']
